In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from sklearn.preprocessing import LabelEncoder
import os
import tensorflow as tf
from tensorflow import keras 
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [12]:
AUTOTUNE = tf.data.AUTOTUNE 

In [16]:
DATA_DIR = '../input/jpeg-happywhale-128x128/train_images-128-128/train_images-128-128/'

In [17]:
train_df = pd.read_csv('../input/happy-whale-and-dolphin/train.csv')
train_df.head()

,image,species,individual_id
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9
1,000562241d384d.jpg,humpback_whale,1a71fbb72250
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b
3,0007d9bca26a99.jpg,bottlenose_dolphin,4b00fe572063
4,00087baf5cef7a.jpg,humpback_whale,8e5253662392


In [18]:
train_df['individual_id'].nunique()

15587

In [19]:
train_df['species'].nunique()

30

In [20]:
label_encoder = LabelEncoder()
train_df['label'] = label_encoder.fit_transform(train_df['individual_id'])
train_df['path'] = DATA_DIR+train_df['image']
train_df.head()

,image,species,individual_id,label,path
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9,12348,../input/jpeg-happywhale-128x128/train_images-...
1,000562241d384d.jpg,humpback_whale,1a71fbb72250,1636,../input/jpeg-happywhale-128x128/train_images-...
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b,5842,../input/jpeg-happywhale-128x128/train_images-...
3,0007d9bca26a99.jpg,bottlenose_dolphin,4b00fe572063,4551,../input/jpeg-happywhale-128x128/train_images-...
4,00087baf5cef7a.jpg,humpback_whale,8e5253662392,8721,../input/jpeg-happywhale-128x128/train_images-...


In [22]:
dataset = tf.data.Dataset.from_tensor_slices((train_df['path'], train_df['label']))

for image, label in dataset.take(1):
    print(image)
    print(label)

tf.Tensor(b'../input/jpeg-happywhale-128x128/train_images-128-128/train_images-128-128/00021adfb725ed.jpg', shape=(), dtype=string)
tf.Tensor(12348, shape=(), dtype=int64)


2022-09-15 17:35:22.456134: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [24]:
def process_path(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    return img, label

def preprocess(image, label):
    input_image = tf.image.resize(image, (128, 128), method='nearest')

    input_image = input_image / 255.

    return input_image, label

image_ds = dataset.map(process_path, num_parallel_calls = AUTOTUNE)
processed_image_ds = image_ds.map(preprocess, num_parallel_calls = AUTOTUNE)

In [25]:
for image, label in processed_image_ds.take(1):
    print(image)
    print(label)

tf.Tensor(
[[[0.00141484 0.00210688 0.00275279]
  [0.00141484 0.00210688 0.00275279]
  [0.00147636 0.00213764 0.00279892]
  ...
  [0.00089196 0.00141484 0.00215302]
  [0.00083045 0.00135333 0.0020915 ]
  [0.00096886 0.00149173 0.00222991]]

 [[0.00103037 0.00163014 0.00226067]
  [0.00106113 0.0016609  0.00229143]
  [0.00098424 0.00158401 0.00221453]
  ...
  [0.00083045 0.00138408 0.00212226]
  [0.00089196 0.0014456  0.00218378]
  [0.00076894 0.00132257 0.00206075]]

 [[0.00163014 0.00221453 0.0028143 ]
  [0.00179931 0.0023837  0.00298347]
  [0.0018762  0.00246059 0.00306036]
  ...
  [0.0009381  0.00150711 0.00219915]
  [0.00099962 0.00156863 0.00226067]
  [0.00103037 0.00159938 0.00227605]]

 ...

 [[0.00073818 0.00073818 0.00089196]
  [0.00066128 0.00066128 0.00081507]
  [0.00063053 0.00063053 0.00078431]
  ...
  [0.00076894 0.0007228  0.00083045]
  [0.00078431 0.00073818 0.00084583]
  [0.00078431 0.00073818 0.00084583]]

 [[0.00070742 0.00070742 0.00086121]
  [0.00069204 0.00069204 0

2022-09-15 17:37:28.221610: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [26]:
dataset = processed_image_ds.shuffle(1024, reshuffle_each_iteration = True)
dataset = dataset.prefetch(AUTOTUNE)

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
IMG_SIZE = (128, 128)

In [ ]:
def whale_model(image_shape=IMG_SIZE):
    ''' Define a tf.keras model for binary classification out of the MobileNetV2 model
    Arguments:
        image_shape -- Image width and height
        data_augmentation -- data augmentation function
    Returns:
    Returns:
        tf.keras.model
    '''
    
    
    input_shape = image_shape + (3,)
    
    base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape=input_shape,
    include_top=False,
    weights='imagenet'
    )
    
    base_model.trainable = False
    inputs = tf.keras.Input(input_shape)
    
    X = preprocess_input(inputs)
    X = base_model(X, training=False)
    
    X = tfl.GlobalAveragePooling2D()(X)
    X = tfl.Dropout(0.2)(X)
    
    prediction_layer = tfl.Dense(15587, activation='softmax')
    
    outputs = prediction_layer(X) 
    model = tf.keras.Model(inputs, outputs)
    return model

In [ ]:
model = whale_model(IMG_SIZE)

In [ ]:
base_learning_rate = 0.01
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.metrics.TopKCategoricalAccuracy(k=5),
              metrics=['accuracy'])